In [1]:
import pandas as pd
import numpy as np
import os
import datetime as dt
from datetime import timedelta
import xgboost as xgb
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from xgboost import XGBClassifier
from matplotlib import pyplot
from xgboost import plot_importance
os.getcwd()
os.chdir("D:\\he_landmark\\Data\\")

In [2]:
os.getcwd()

'D:\\he_landmark\\Data'

In [3]:
custDemograph = pd.read_excel('Customer_Demographics.xlsx')
custTrans = pd.read_excel('Customer_Transaction.xlsx')
storeMas = pd.read_excel('Store_Master.xlsx')
test = pd.read_excel('Test_Set.xlsx')

In [4]:
custDemograph.shape,custTrans.shape,storeMas.shape,test.shape

((100000, 16), (544649, 15), (35, 15), (400000, 2))

In [50]:
custDemograph.head() #custDemograph.columns

,Customer_ID,Territory,Nationality,Income_Range,Job_Type,Marital_Status,Gender,State,Language,Loyalty_Status,Birth_date,Age,Points,First_txn_dt,Last_accr_txn_dt,Last_rdm_txn_dt
0,1800000006365760,United Arab Emirates,INDIA,Below 5000,Services,Married,M,Abu Dhabi,English,Gold,01JUN1959:00:00:00,59.0,814.0,02NOV2010:00:00:00,27SEP2018:21:45:25,11MAR2018:19:11:57
1,1800000006365820,United Arab Emirates,UNITED ARAB EMIRATES,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,01JAN1900:00:00:00,NaN,212.0,06DEC2010:00:00:00,16AUG2016:21:08:28,16MAR2015:13:39:12
2,1800000006366060,United Arab Emirates,SRI LANKA,Below 5000,Services,Married,M,Abu Dhabi,English,Silver,07JUN1959:00:00:00,59.0,186.0,17FEB2011:00:00:00,31AUG2018:20:52:18,30JAN2017:19:19:19
3,1800000006366230,United Arab Emirates,JORDAN,Below 5000,Services,Married,F,Abu Dhabi,Arabic,Silver,21JAN1962:00:00:00,57.0,126.0,30OCT2010:00:00:00,30JUL2018:20:30:05,20JUN2017:17:57:27
4,1800000006366570,United Arab Emirates,OCCUPIED PALESTINIAN TERRITORY,Below 5000,Services,Married,M,Abu Dhabi,Arabic,Silver,01JAN1900:00:00:00,NaN,197.0,10NOV2010:00:00:00,03SEP2018:22:10:56,17JUN2017:23:08:28


In [15]:
custTrans[custTrans['Units_Sold'] < 0].shape,custTrans.shape

((36918, 15), (544649, 15))

In [23]:
from __future__ import division
36918/544649*100


6.778310434793784

In [24]:
custTrans.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,Invoices,Item_Count,Revenue,Discount,Units_Sold
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,1,3,-169.0,0.0,-3
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,1,1,0.0,0.0,0
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,1,1,0.0,0.0,0
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,1,4,0.0,0.0,0
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,1,1,0.0,0.0,0


In [188]:
custMax=custTrans.groupby("Customer_ID").agg({'Revenue' : ['size','sum','max','min']}).reset_index()
custMax.columns = custMax.columns.droplevel(0)
custMax.columns = ['Customer_ID' , 'revenuesize' , 'revenuesum','revenuemax','revenuemin']

In [189]:
custMax2=custTrans.groupby("Customer_ID").agg({'Item_Count' : ['size','sum','max','min']}).reset_index()
custMax2.columns = custMax2.columns.droplevel(0)
custMax2.columns = ['Customer_ID' , 'itemsize' , 'itemsum','itemmax','itemmin']

In [190]:
custMax3=custTrans.groupby("Customer_ID").agg({'Discount' : ['size','sum','max','min']}).reset_index()
custMax3.columns = custMax3.columns.droplevel(0)
custMax3.columns = ['Customer_ID' , 'dissize' , 'dissum','dismax','dismin']

In [191]:
custMax4=custTrans.groupby("Customer_ID").agg({'Units_Sold' : ['size','sum','max','min']}).reset_index()
custMax4.columns = custMax4.columns.droplevel(0)
custMax4.columns = ['Customer_ID' , 'unitssize' , 'unitssum','unitsmax','unitsmin']

In [192]:
custMax5=custTrans.groupby("Customer_ID").agg({'Invoices' : ['size','sum','max','min']}).reset_index()
custMax5.columns = custMax5.columns.droplevel(0)
custMax5.columns = ['Customer_ID' , 'invoicesize' , 'invoiceesum','invoicemax','invoicemin']

In [193]:
storeNew1=custTrans.groupby("Store_Code").agg({'Revenue' : ['size','sum','max','min']}).reset_index()
storeNew1.columns = storeNew1.columns.droplevel(0)
storeNew1.columns = ['Store_Code' , 'storerevenuesize' , 'storerevenuesum','storerevenuemax','storerevenuemin']

In [194]:
storeNew2=custTrans.groupby("Store_Code").agg({'Discount' : ['size','sum','max','min']}).reset_index()
storeNew2.columns = storeNew2.columns.droplevel(0)
storeNew2.columns = ['Store_Code' , 'storedissize' , 'storedissum','storedismax','storedismin']

In [195]:
storeNew3=custTrans.groupby("Store_Code").agg({'Item_Count' : ['size','sum','max','min']}).reset_index()
storeNew3.columns = storeNew3.columns.droplevel(0)
storeNew3.columns = ['Store_Code' , 'storeItemcount' , 'storeitemsum','storeitemmax','storeitemmin']

In [196]:
storeNew4=custTrans.groupby("Store_Code").agg({'Units_Sold' : ['size','sum','max','min']}).reset_index()
storeNew4.columns = storeNew4.columns.droplevel(0)
storeNew4.columns = ['Store_Code' , 'storeunits' , 'storeunitssum','storeunitsmax','storeunitsmin']

In [57]:
custMax.head()

,Customer_ID,revenuesize,revenuesum,revenuemax,revenuemin
0,1800000006365760,4,1117.00,850.0,34.0
1,1800000006365820,1,260.00,260.0,260.0
2,1800000006366060,6,607.00,188.0,75.0
3,1800000006366230,6,643.80,151.0,54.0
4,1800000006366570,10,1285.01,277.0,-105.0


In [4]:
transacCusts=custTrans['Customer_ID'].unique()

In [7]:
storeMas[storeMas['Store_Code'].isin([60065,60298,60301,60053])]

,Territory,Business,Region,Region_Code,Store_Code,Store_Name,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue
2,United Arab Emirates,Max,MX - Dubai,601,60065,Max Gardens,Ibn Battuta Mall-Uae,NaN,3319.885702,2005-04-15,Train,"25.044684, 55.120275",15609.687991,188836.0,69053622.61
28,United Arab Emirates,Max,MX - Abu Dhabi,602,60053,Max Abudhabi,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49
31,United Arab Emirates,Max,MX - Abu Dhabi,602,60301,MAX- Abudhabi Al Wahda,AL Wahda,Mall Stores,10861.009520,2018-01-30,Test,"24.470587, 54.372679",23288.338616,NaN,7472374.55
32,United Arab Emirates,Max,MX - Sharjah,606,60298,MAX - Matejer Al Juraina,Matejer Al Juraina,Mall Stores,4099.543401,2018-01-07,Test,"25.296484, 55.489202",8391.589347,NaN,2820485.86


In [11]:
storeMas.shape

(35, 15)

In [9]:
leftout=test[test['Customer_ID'].isin(transacCusts)]

In [10]:
leftout.shape,test.shape

((400000, 2), (400000, 2))

In [8]:
test.head()

,Customer_ID,Store_Code
0,1800000006365760,60298
1,1800000006365760,60299
2,1800000006365760,60300
3,1800000006365760,60301
4,1800000006365820,60298


In [25]:
storeMas.Store_Format.unique()

array([nan, u'Stand Alone Stores', u'Mall Stores'], dtype=object)

In [26]:
custTotal=custTrans.merge(custDemograph[['Customer_ID','Nationality','Income_Range','Job_Type','Marital_Status',
                                         'Gender','State', u'Language','Loyalty_Status','Birth_date','Age','Points',
                                         'First_txn_dt','Last_accr_txn_dt', 'Last_rdm_txn_dt']],how='left',on='Customer_ID')

In [27]:
custTotal.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,...,Gender,State,Language,Loyalty_Status,Birth_date,Age,Points,First_txn_dt,Last_accr_txn_dt,Last_rdm_txn_dt
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,...,F,Dubai,Arabic,Silver,25JAN1981:00:00:00,38.0,276.0,06MAY2013:00:00:00,19AUG2018:22:38:24,10MAY2018:21:28:38
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,...,M,Abu Dhabi,English,Silver,22JAN1971:00:00:00,48.0,78.0,09FEB2011:00:00:00,26SEP2018:17:40:53,21JUL2018:22:38:53
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,...,M,Abu Dhabi,English,Gold,31OCT1966:00:00:00,52.0,640.0,17NOV2011:00:00:00,16SEP2018:16:23:03,15SEP2018:12:19:40
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,...,F,Abu Dhabi,English,Silver,25APR1975:00:00:00,43.0,1205.0,16JUL2011:00:00:00,01SEP2018:10:43:50,01AUG2013:18:08:40
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,...,F,Abu Dhabi,English,Silver,19NOV1967:00:00:00,51.0,180.0,12FEB2013:00:00:00,26SEP2018:17:15:35,09SEP2018:18:12:21


In [28]:
import datetime as dt
def weekday(x):
    try:
        return dt.datetime.strptime(x,'%d%b%Y:%H:%M:%S').weekday()
    except:
        return np.nan

def day(x):
    try:
        return dt.datetime.strptime(x,'%d%b%Y:%H:%M:%S').day
    except:
        return np.nan

In [29]:
custTotal['First_txn_day']=custTotal['First_txn_dt'].apply(lambda x: day(x))
custTotal['First_txn_weekday']=custTotal['First_txn_dt'].apply(lambda x: weekday(x))

In [30]:
custTotal.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,...,Language,Loyalty_Status,Birth_date,Age,Points,First_txn_dt,Last_accr_txn_dt,Last_rdm_txn_dt,First_txn_day,First_txn_weekday
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,...,Arabic,Silver,25JAN1981:00:00:00,38.0,276.0,06MAY2013:00:00:00,19AUG2018:22:38:24,10MAY2018:21:28:38,6.0,0.0
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,...,English,Silver,22JAN1971:00:00:00,48.0,78.0,09FEB2011:00:00:00,26SEP2018:17:40:53,21JUL2018:22:38:53,9.0,2.0
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,...,English,Gold,31OCT1966:00:00:00,52.0,640.0,17NOV2011:00:00:00,16SEP2018:16:23:03,15SEP2018:12:19:40,17.0,3.0
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,...,English,Silver,25APR1975:00:00:00,43.0,1205.0,16JUL2011:00:00:00,01SEP2018:10:43:50,01AUG2013:18:08:40,16.0,5.0
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,...,English,Silver,19NOV1967:00:00:00,51.0,180.0,12FEB2013:00:00:00,26SEP2018:17:15:35,09SEP2018:18:12:21,12.0,1.0


In [36]:
storeMas.columns

Index([u'Territory', u'Business', u'Region', u'Region_Code', u'Store_Code',
       u'Store_Name', u'Mall_Name', u'Store_Format', u'Sales_Per_Day',
       u'Store_Launch_Date', u'Train_Test_Store', u'Geo_Field',
       u'Store_Size_Sq_Ft', u'Customer_Count', u'Total_Revenue'],
      dtype='object')

In [179]:
storeMas.Store_Format.unique()

array([nan, u'Stand Alone Stores', u'Mall Stores'], dtype=object)

In [37]:
custTotal_store=custTotal.merge(storeMas[['Region', u'Region_Code', u'Store_Code',
       u'Store_Name', u'Mall_Name', u'Store_Format', u'Sales_Per_Day',
       u'Store_Launch_Date', u'Train_Test_Store', u'Geo_Field',
       u'Store_Size_Sq_Ft', u'Customer_Count', u'Total_Revenue']],how='left',on='Store_Code')

In [38]:
custTotal_store.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,...,Store_Name,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,...,Max Gardens,Ibn Battuta Mall-Uae,NaN,3319.885702,2005-04-15,Train,"25.044684, 55.120275",15609.687991,188836.0,69053622.61
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,...,Max Abudhabi,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,...,Max Abudhabi,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,...,Max Abudhabi,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,...,Max Abudhabi,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49


In [39]:
def purchaseFlag(x):
    if x < 0:
        return 0
    else:
        return 1

In [40]:
custTotal_store['purchaseFlag']=custTotal_store['Units_Sold'].apply(lambda x:purchaseFlag(x))

In [41]:
custTotal_store.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,...,Mall_Name,Store_Format,Sales_Per_Day,Store_Launch_Date,Train_Test_Store,Geo_Field,Store_Size_Sq_Ft,Customer_Count,Total_Revenue,purchaseFlag
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,...,Ibn Battuta Mall-Uae,NaN,3319.885702,2005-04-15,Train,"25.044684, 55.120275",15609.687991,188836.0,69053622.61,0
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,...,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49,1
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,...,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49,1
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,...,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49,1
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,...,STANDALONE,NaN,3094.424495,2004-05-16,Train,NaN,19200.122169,160218.0,64364029.49,1


In [288]:
custTotal_store2=custTotal_store.merge(custMax,how='left',on='Customer_ID')
custTotal_store2=custTotal_store2.merge(custMax2,how='left',on='Customer_ID')
custTotal_store2=custTotal_store2.merge(custMax3,how='left',on='Customer_ID')
custTotal_store2=custTotal_store2.merge(custMax4,how='left',on='Customer_ID')
custTotal_store2=custTotal_store2.merge(custMax5,how='left',on='Customer_ID')
#custTotal_store2=custTotal_store2.merge(storeNew1,how='left',on='Store_Code')
#custTotal_store2=custTotal_store2.merge(storeNew2,how='left',on='Store_Code')
#custTotal_store2=custTotal_store2.merge(storeNew3,how='left',on='Store_Code')
#custTotal_store2=custTotal_store2.merge(storeNew4,how='left',on='Store_Code')


In [289]:
custTotal_store2.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,...,dismax,dismin,unitssize,unitssum,unitsmax,unitsmin,invoicesize,invoiceesum,invoicemax,invoicemin
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,...,697.0,-39.0,24,138,46,-5,24,26,2,1
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,...,493.0,-29.0,17,88,32,-1,17,21,3,1
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,...,140.0,-22.0,57,259,26,-8,57,68,3,1
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,...,10.0,0.0,5,19,10,-1,5,6,2,1
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,...,161.0,0.0,9,75,21,-2,9,10,2,1


In [290]:
custTotal_store2.isnull().sum()

Territory                 0
Business                  0
Year                      0
Week                      0
Store_Code                0
City_Name             39185
Store_Type            39185
Transaction_Type          0
Return_Reason        507635
Customer_ID               0
Invoices                  0
Item_Count                0
Revenue                   0
Discount                  0
Units_Sold                0
Nationality               0
Income_Range              0
Job_Type                  0
Marital_Status            0
Gender                    0
State                     0
Language                  0
Loyalty_Status            0
Birth_date            53003
Age                   89003
Points                 3997
First_txn_dt             59
Last_accr_txn_dt          7
Last_rdm_txn_dt       60568
First_txn_day            59
                      ...  
Mall_Name                 1
Store_Format         485708
Sales_Per_Day             1
Store_Launch_Date         1
Train_Test_Store    

In [364]:
custTotal_store2[['City_Name','Store_Type','Birth_date','Age','Store_Format','First_txn_dt','Last_accr_txn_dt','First_txn_day','First_txn_weekday']]=custTotal_store2[['City_Name','Store_Type','Birth_date','Age','Store_Format','First_txn_dt','Last_accr_txn_dt','First_txn_day','First_txn_weekday']].fillna(value='other')
custTotal_store3=custTotal_store2.drop(['Last_rdm_txn_dt','Return_Reason','Geo_Field','First_txn_dt','Last_accr_txn_dt','Birth_date','Year','Week','Customer_Count','Store_Launch_Date','City_Name','Store_Type','Transaction_Type','Invoices','Item_Count','Revenue','Discount','Units_Sold'],axis = 1)

In [365]:
'''Year
Week
City_Name
Store_Type
Transaction_Type
Invoices
Item_Count
Revenue
Discount
Units_Sold'''


'Year\nWeek\nCity_Name\nStore_Type\nTransaction_Type\nInvoices\nItem_Count\nRevenue\nDiscount\nUnits_Sold'

In [366]:
custTotal_store4=custTotal_store3.dropna()
custTotal_store4.isnull().sum()
custTotal_store4.dtypes

Territory             object
Business              object
Store_Code             int64
Customer_ID            int64
Nationality           object
Income_Range          object
Job_Type              object
Marital_Status        object
Gender                object
State                 object
Language              object
Loyalty_Status        object
Age                   object
Points               float64
First_txn_day         object
First_txn_weekday     object
Region                object
Region_Code          float64
Store_Name            object
Mall_Name             object
Store_Format          object
Sales_Per_Day        float64
Train_Test_Store      object
Store_Size_Sq_Ft     float64
Total_Revenue        float64
purchaseFlag           int64
revenuesize            int64
revenuesum           float64
revenuemax           float64
revenuemin           float64
itemsize               int64
itemsum                int64
itemmax                int64
itemmin                int64
dissize       

In [297]:
custTotal_store3.isnull().sum()


Territory               0
Business                0
Store_Code              0
Customer_ID             0
Nationality             0
Income_Range            0
Job_Type                0
Marital_Status          0
Gender                  0
State                   0
Language                0
Loyalty_Status          0
Age                     0
Points               3997
First_txn_day           0
First_txn_weekday       0
Region                  1
Region_Code             1
Store_Name              1
Mall_Name               1
Store_Format            0
Sales_Per_Day           1
Train_Test_Store        1
Store_Size_Sq_Ft        1
Customer_Count          1
Total_Revenue           1
purchaseFlag            0
revenuesize             0
revenuesum              0
revenuemax              0
revenuemin              0
itemsize                0
itemsum                 0
itemmax                 0
itemmin                 0
dissize                 0
dissum                  0
dismax                  0
dismin      

In [298]:
#custTotal_store3['Points']=pd.to_numeric(custTotal_store3['Points'])

In [367]:
#char_cols = custTotal_store4.dtypes.pipe(lambda x: x[x == 'object']).index

#custTotal_store5 = pd.get_dummies(custTotal_store4,prefix=char_cols)

In [383]:
char_colsNw = totaldata.dtypes.pipe(lambda x: x[x == 'object']).index

totaldata2 = pd.get_dummies(totaldata,prefix=char_colsNw)

In [384]:
custTotal_store5.shape

(540651, 516)

In [388]:
custTotal_store6=totaldata2[totaldata2['purchaseFlag']!=-1]
testNw=totaldata2[totaldata2['purchaseFlag']==-1]

In [389]:
custTotal_store6.shape,custTotal_store5.shape,test3.shape,custTotal_store4.shape,testNw.shape

((540651, 529), (540651, 516), (400000, 46), (540651, 46), (400000, 529))

In [390]:
custTotal_store6.columns

Index([                       u'Store_Code',
                             u'Customer_ID',
                                  u'Points',
                             u'Region_Code',
                           u'Sales_Per_Day',
                        u'Store_Size_Sq_Ft',
                           u'Total_Revenue',
                            u'purchaseFlag',
                             u'revenuesize',
                              u'revenuesum',
       ...
       u'Mall_Name_Safeer Mall Sharjah-Uae',
       u'Mall_Name_Sharjah City Centre-Uae',
             u'Mall_Name_Shindaga Mall-Uae',
                           u'Mall_Name_WTC',
                  u'Mall_Name_Yas Mall-Uae',
                u'Store_Format_Mall Stores',
         u'Store_Format_Stand Alone Stores',
                      u'Store_Format_other',
                   u'Train_Test_Store_Test',
                  u'Train_Test_Store_Train'],
      dtype='object', length=529)

In [368]:
for col in char_cols:
    print(col,len(custTotal_store4[col].unique()))

(u'Territory', 1)
(u'Business', 1)
(u'Nationality', 222)
(u'Income_Range', 7)
(u'Job_Type', 19)
(u'Marital_Status', 7)
(u'Gender', 3)
(u'State', 13)
(u'Language', 3)
(u'Loyalty_Status', 2)
(u'Age', 104)
('First_txn_day', 32)
('First_txn_weekday', 8)
(u'Region', 6)
(u'Store_Name', 31)
(u'Mall_Name', 25)
(u'Store_Format', 3)
(u'Train_Test_Store', 1)


In [301]:
custTotal_store3[char_cols[0]].unique()

array([u'United Arab Emirates'], dtype=object)

In [302]:
custTotal_store4.isnull().sum()

Territory            0
Business             0
Store_Code           0
Customer_ID          0
Nationality          0
Income_Range         0
Job_Type             0
Marital_Status       0
Gender               0
State                0
Language             0
Loyalty_Status       0
Age                  0
Points               0
First_txn_day        0
First_txn_weekday    0
Region               0
Region_Code          0
Store_Name           0
Mall_Name            0
Store_Format         0
Sales_Per_Day        0
Train_Test_Store     0
Store_Size_Sq_Ft     0
Customer_Count       0
Total_Revenue        0
purchaseFlag         0
revenuesize          0
revenuesum           0
revenuemax           0
revenuemin           0
itemsize             0
itemsum              0
itemmax              0
itemmin              0
dissize              0
dissum               0
dismax               0
dismin               0
unitssize            0
unitssum             0
unitsmax             0
unitsmin             0
invoicesize

In [303]:
custTotal_store5.dtypes

Store_Code                                  int64
Customer_ID                                 int64
Points                                    float64
Region_Code                               float64
Sales_Per_Day                             float64
Store_Size_Sq_Ft                          float64
Customer_Count                            float64
Total_Revenue                             float64
purchaseFlag                                int64
revenuesize                                 int64
revenuesum                                float64
revenuemax                                float64
revenuemin                                float64
itemsize                                    int64
itemsum                                     int64
itemmax                                     int64
itemmin                                     int64
dissize                                     int64
dissum                                    float64
dismax                                    float64


In [97]:
df = pd.DataFrame({'A': ['a', 'b', 'a'], 'B': ['b', 'a', 'c'],'C': [1, 2, 3]})
df

,A,B,C
0,a,b,1
1,b,a,2
2,a,c,3


In [98]:
 pd.get_dummies(df, prefix=['col1', 'col2'])

,C,col1_a,col1_b,col2_a,col2_b,col2_c
0,1,1,0,0,1,0
1,2,0,1,1,0,0
2,3,1,0,0,0,1


In [304]:
custTotal_store5=custTotal_store5.reset_index()
custTotal_store5.columns


Index([                             u'index',
                               u'Store_Code',
                              u'Customer_ID',
                                   u'Points',
                              u'Region_Code',
                            u'Sales_Per_Day',
                         u'Store_Size_Sq_Ft',
                           u'Customer_Count',
                            u'Total_Revenue',
                             u'purchaseFlag',
       ...
                     u'Mall_Name_STANDALONE',
       u'Mall_Name_Safeer Mall - Rakuae-Uae',
        u'Mall_Name_Safeer Mall Sharjah-Uae',
        u'Mall_Name_Sharjah City Centre-Uae',
              u'Mall_Name_Shindaga Mall-Uae',
                   u'Mall_Name_Yas Mall-Uae',
                 u'Store_Format_Mall Stores',
          u'Store_Format_Stand Alone Stores',
                       u'Store_Format_other',
                   u'Train_Test_Store_Train'],
      dtype='object', length=518)

In [305]:
#del custTotal_store5['level_0']
del custTotal_store5['index']
#del custTotal_store5[u'index']

custTotal_store5.columns


Index([                        u'Store_Code',
                              u'Customer_ID',
                                   u'Points',
                              u'Region_Code',
                            u'Sales_Per_Day',
                         u'Store_Size_Sq_Ft',
                           u'Customer_Count',
                            u'Total_Revenue',
                             u'purchaseFlag',
                              u'revenuesize',
       ...
                     u'Mall_Name_STANDALONE',
       u'Mall_Name_Safeer Mall - Rakuae-Uae',
        u'Mall_Name_Safeer Mall Sharjah-Uae',
        u'Mall_Name_Sharjah City Centre-Uae',
              u'Mall_Name_Shindaga Mall-Uae',
                   u'Mall_Name_Yas Mall-Uae',
                 u'Store_Format_Mall Stores',
          u'Store_Format_Stand Alone Stores',
                       u'Store_Format_other',
                   u'Train_Test_Store_Train'],
      dtype='object', length=517)

In [391]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(custTotal_store6.drop(['Store_Code','Customer_ID'],axis=1) ,custTotal_store6['purchaseFlag'],
                                test_size=0.33, random_state=42,shuffle=True,stratify=custTotal_store6['purchaseFlag'])

In [392]:
yw=custTotal_store3['purchaseFlag']

In [393]:
from collections import Counter
Counter(yw)


Counter({0: 36918, 1: 507731})

In [394]:
custTotal_store2.shape

(544649, 64)

In [395]:
from sklearn.model_selection import train_test_split

In [396]:
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((362236, 527), (178415, 527), (362236L,), (178415L,))

In [397]:
X_train.isnull().sum()

Points                                    0
Region_Code                               0
Sales_Per_Day                             0
Store_Size_Sq_Ft                          0
Total_Revenue                             0
purchaseFlag                              0
revenuesize                               0
revenuesum                                0
revenuemax                                0
revenuemin                                0
itemsize                                  0
itemsum                                   0
itemmax                                   0
itemmin                                   0
dissize                                   0
dissum                                    0
dismax                                    0
dismin                                    0
unitssize                                 0
unitssum                                  0
unitsmax                                  0
unitsmin                                  0
invoicesize                     

In [406]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.datasets import make_classification
clf = RandomForestClassifier(n_estimators=500, max_depth=5,random_state=1)
clf.fit(X_train, y_train)
print(clf.feature_importances_)

[  3.87323667e-04   2.87127075e-03   1.91640847e-03   8.18090859e-04
   1.51472335e-03   4.29386373e-01   1.79700748e-02   7.40946173e-03
   6.88287764e-03   1.27733544e-01   1.32091651e-02   9.21412413e-03
   8.15352992e-03   1.51639164e-02   1.52477851e-02   6.48122413e-03
   6.10702165e-03   7.44497615e-02   1.56803537e-02   7.78615419e-03
   8.50072986e-03   1.35356807e-01   1.82783414e-02   1.92742874e-02
   1.78716476e-02   1.09810582e-04   0.00000000e+00   0.00000000e+00
   3.51566836e-06   1.21955937e-07   1.74776460e-05   2.84594490e-06
   0.00000000e+00   3.74611734e-06   0.00000000e+00   1.78969423e-06
   0.00000000e+00   1.67753719e-06   3.54158175e-06   0.00000000e+00
   1.36052607e-05   5.01425659e-07   0.00000000e+00   0.00000000e+00
   3.01849071e-07   8.13433993e-06   0.00000000e+00   1.52581829e-06
   1.09210294e-06   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   8.67984839e-08   0.00000000e+00
   0.00000000e+00   1.61965832e-05

In [407]:
from sklearn.model_selection import cross_val_score
cross_val_score(clf,X_train,y_train,cv=10)

array([ 0.9320064 ,  0.9320064 ,  0.9320064 ,  0.9320064 ,  0.9320064 ,
        0.9320064 ,  0.9320064 ,  0.93203213,  0.93203213,  0.93203026])

In [410]:
feature_set1=list(custTotal_store6.columns)
feature_set1.remove('Store_Code')
feature_set1.remove('Customer_ID')

In [420]:
train_X1 = custTotal_store6[feature_set1]
test_X1 = testNw[feature_set1]


train_X1 = train_X1.as_matrix()
test_X1 = test_X1.as_matrix()

train_Y = custTotal_store6['purchaseFlag']

mask = np.random.choice([False, True], len(train_X1), p=[0.7, 0.3])
not_mask = ~mask

In [421]:
xg_train1 = xgb.DMatrix(train_X1[not_mask], label=train_Y[not_mask],feature_names=feature_set1)
xg_val1 = xgb.DMatrix(train_X1[mask], label=train_Y[mask],feature_names=feature_set1)
xg_test1 = xgb.DMatrix(test_X1,feature_names=feature_set1)

param = {}
param['objective'] = 'binary:logistic'
param['eta'] = 0.01
param['max_depth'] = 6
param['silent'] = 1
param['eval_metric'] = "auc"
param['subsample'] = 0.5
param['colsample_bytree'] = 0.5

In [422]:
evallist1  = [(xg_val1, 'eval')]
bst1 = xgb.train( param, xg_train1, 1000, evallist1 )

[0]	eval-auc:0.835922
[1]	eval-auc:1
[2]	eval-auc:1
[3]	eval-auc:1
[4]	eval-auc:1
[5]	eval-auc:1
[6]	eval-auc:1
[7]	eval-auc:1
[8]	eval-auc:1
[9]	eval-auc:1
[10]	eval-auc:1
[11]	eval-auc:1
[12]	eval-auc:1
[13]	eval-auc:1
[14]	eval-auc:1
[15]	eval-auc:1
[16]	eval-auc:1
[17]	eval-auc:1
[18]	eval-auc:1
[19]	eval-auc:1
[20]	eval-auc:1
[21]	eval-auc:1
[22]	eval-auc:1
[23]	eval-auc:1
[24]	eval-auc:1
[25]	eval-auc:1
[26]	eval-auc:1
[27]	eval-auc:1
[28]	eval-auc:1
[29]	eval-auc:1
[30]	eval-auc:1
[31]	eval-auc:1
[32]	eval-auc:1
[33]	eval-auc:1
[34]	eval-auc:1
[35]	eval-auc:1
[36]	eval-auc:1
[37]	eval-auc:1
[38]	eval-auc:1
[39]	eval-auc:1
[40]	eval-auc:1
[41]	eval-auc:1
[42]	eval-auc:1
[43]	eval-auc:1
[44]	eval-auc:1
[45]	eval-auc:1
[46]	eval-auc:1
[47]	eval-auc:1
[48]	eval-auc:1
[49]	eval-auc:1
[50]	eval-auc:1
[51]	eval-auc:1
[52]	eval-auc:1
[53]	eval-auc:1
[54]	eval-auc:1
[55]	eval-auc:1
[56]	eval-auc:1
[57]	eval-auc:1
[58]	eval-auc:1
[59]	eval-auc:1
[60]	eval-auc:1
[61]	eval-auc:1
[62]	eval-a

[488]	eval-auc:1
[489]	eval-auc:1
[490]	eval-auc:1
[491]	eval-auc:1
[492]	eval-auc:1
[493]	eval-auc:1
[494]	eval-auc:1
[495]	eval-auc:1
[496]	eval-auc:1
[497]	eval-auc:1
[498]	eval-auc:1
[499]	eval-auc:1
[500]	eval-auc:1
[501]	eval-auc:1
[502]	eval-auc:1
[503]	eval-auc:1
[504]	eval-auc:1
[505]	eval-auc:1
[506]	eval-auc:1
[507]	eval-auc:1
[508]	eval-auc:1
[509]	eval-auc:1
[510]	eval-auc:1
[511]	eval-auc:1
[512]	eval-auc:1
[513]	eval-auc:1
[514]	eval-auc:1
[515]	eval-auc:1
[516]	eval-auc:1
[517]	eval-auc:1
[518]	eval-auc:1
[519]	eval-auc:1
[520]	eval-auc:1
[521]	eval-auc:1
[522]	eval-auc:1
[523]	eval-auc:1
[524]	eval-auc:1
[525]	eval-auc:1
[526]	eval-auc:1
[527]	eval-auc:1
[528]	eval-auc:1
[529]	eval-auc:1
[530]	eval-auc:1
[531]	eval-auc:1
[532]	eval-auc:1
[533]	eval-auc:1
[534]	eval-auc:1
[535]	eval-auc:1
[536]	eval-auc:1
[537]	eval-auc:1
[538]	eval-auc:1
[539]	eval-auc:1
[540]	eval-auc:1
[541]	eval-auc:1
[542]	eval-auc:1
[543]	eval-auc:1
[544]	eval-auc:1
[545]	eval-auc:1
[546]	eval-auc

[970]	eval-auc:1
[971]	eval-auc:1
[972]	eval-auc:1
[973]	eval-auc:1
[974]	eval-auc:1
[975]	eval-auc:1
[976]	eval-auc:1
[977]	eval-auc:1
[978]	eval-auc:1
[979]	eval-auc:1
[980]	eval-auc:1
[981]	eval-auc:1
[982]	eval-auc:1
[983]	eval-auc:1
[984]	eval-auc:1
[985]	eval-auc:1
[986]	eval-auc:1
[987]	eval-auc:1
[988]	eval-auc:1
[989]	eval-auc:1
[990]	eval-auc:1
[991]	eval-auc:1
[992]	eval-auc:1
[993]	eval-auc:1
[994]	eval-auc:1
[995]	eval-auc:1
[996]	eval-auc:1
[997]	eval-auc:1
[998]	eval-auc:1
[999]	eval-auc:1


In [423]:
preds1 = bst1.predict(xg_test1)


In [ ]:
preds2=[]
for i in preds1:
    if i > .01:
        preds2.append(1)
    else:
        preds2.append(0)

In [449]:
prediction1 = pd.DataFrame(preds2)
prediction1.columns = ['Prediction']

In [450]:
len(preds2),preds1.shape

(400000, (400000L,))

In [459]:
Submit1 = test.iloc[:,:]
submit1 = pd.concat([Submit1, prediction1], axis=1)
submit1.columns
submit1.head()

,Customer_ID,Store_Code,Prediction
0,1800000006365760,60298,1
1,1800000006365760,60299,1
2,1800000006365760,60300,1
3,1800000006365760,60301,1
4,1800000006365820,60298,1


In [460]:
submit1.to_csv("D:\\he_landmark\\submit1.csv",index=None)

In [456]:
prediction1.head()

,Prediction
0,1
1,1
2,1
3,1
4,1


In [457]:
#Submit1 = test.iloc[:,:]
#Submit1.head()

,Customer_ID,Store_Code
0,1800000006365760,60298
1,1800000006365760,60299
2,1800000006365760,60300
3,1800000006365760,60301
4,1800000006365820,60298


In [446]:
preds2=[]
for i in preds1:
    if i > .01:
        preds2.append(1)
    else:
        preds2.append(0)

In [442]:
preds1.mean(),preds1.min(),preds1.max()

(0.16423737, 0.0025833161, 0.25230527)

In [438]:
predstrain = bst1.predict(xgb.DMatrix(train_X1,feature_names=feature_set1))


In [439]:
xg_train1.num_row(),predstrain.shape,train_Y.shape

(378467L, (540651L,), (540651L,))

In [440]:
from sklearn.metrics import roc_curve, auc
import pylab as pl


In [441]:
fpr, tpr, thresholds =roc_curve(train_Y, predstrain)
roc_auc = auc(fpr, tpr)
print("Area under the ROC curve : %f" % roc_auc)

####################################
# The optimal cut off would be where tpr is high and fpr is low
# tpr - (1-fpr) is zero or near to zero is the optimal cut off point
####################################
i = np.arange(len(tpr)) # index for df
roc = pd.DataFrame({'fpr' : pd.Series(fpr, index=i),'tpr' : pd.Series(tpr, index = i), '1-fpr' : pd.Series(1-fpr, index = i), 'tf' : pd.Series(tpr - (1-fpr), index = i), 'thresholds' : pd.Series(thresholds, index = i)})
roc.ix[(roc.tf-0).abs().argsort()[:1]]

# Plot tpr vs 1-fpr
fig, ax = pl.subplots()
pl.plot(roc['tpr'])
pl.plot(roc['1-fpr'], color = 'red')
pl.xlabel('1-False Positive Rate')
pl.ylabel('True Positive Rate')
pl.title('Receiver operating characteristic')
ax.set_xticklabels([])


Area under the ROC curve : 1.000000


C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:11: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  # This is added back by InteractiveShellApp.init_path()


[]

In [424]:
test.shape,prediction1.shape,preds1.shape

((400000, 2), (400000, 1), (400000L,))

In [428]:
preds1.mean()

0.16423737

In [315]:
custTrans.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,Invoices,Item_Count,Revenue,Discount,Units_Sold
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,1,3,-169.0,0.0,-3
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,1,1,0.0,0.0,0
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,1,1,0.0,0.0,0
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,1,4,0.0,0.0,0
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,1,1,0.0,0.0,0


In [316]:
r=custTrans[custTrans['Store_Code']==60298]

In [317]:
r.shape

(0, 15)

In [318]:
test.head()

,Customer_ID,Store_Code
0,1800000006365760,60298
1,1800000006365760,60299
2,1800000006365760,60300
3,1800000006365760,60301
4,1800000006365820,60298


In [319]:
test2=test.merge(custMax,how='left',on='Customer_ID')
test2=test2.merge(custDemograph[['Customer_ID','Nationality','Income_Range','Job_Type','Marital_Status',
                                         'Gender','State', u'Language','Loyalty_Status','Birth_date','Age','Points',
                                         'First_txn_dt','Last_accr_txn_dt', 'Last_rdm_txn_dt']],how='left',on='Customer_ID')
test2=test2.merge(storeMas[[u'Territory',u'Business','Region', u'Region_Code', u'Store_Code',
       u'Store_Name', u'Mall_Name', u'Store_Format', u'Sales_Per_Day',
       u'Store_Launch_Date', u'Train_Test_Store', u'Geo_Field',
       u'Store_Size_Sq_Ft', u'Customer_Count', u'Total_Revenue']],how='left',on='Store_Code')

test2['First_txn_day']=test2['First_txn_dt'].apply(lambda x: day(x))
test2['First_txn_weekday']=test2['First_txn_dt'].apply(lambda x: weekday(x))

In [320]:
#test2=test2.merge(custMax,how='left',on='Customer_ID')
test2=test2.merge(custMax2,how='left',on='Customer_ID')
test2=test2.merge(custMax3,how='left',on='Customer_ID')
test2=test2.merge(custMax4,how='left',on='Customer_ID')
test2=test2.merge(custMax5,how='left',on='Customer_ID')
#test2=test2.merge(storeNew1,how='left',on='Store_Code')
#test2=test2.merge(storeNew2,how='left',on='Store_Code')
#test2=test2.merge(storeNew3,how='left',on='Store_Code')
#test2=test2.merge(storeNew4,how='left',on='Store_Code')


In [321]:
#test2.isnull().sum(),custTotal_store3.isnull().sum()

In [369]:
for i in custTotal_store4.columns:
    if i not in test3.columns:
        print i

purchaseFlag


In [370]:
neededColumns=list(custTotal_store4.columns)

In [372]:
neededColumns.remove('purchaseFlag')

In [373]:
test3=test2[neededColumns].reset_index()

In [374]:
test3['purchaseFlag']=-1

In [376]:
test3=test3.fillna(value=-1)
#test3=test3.drop(['Customer_Count'],axis=1)
test3=test3.drop(['index'],axis=1)

In [377]:
test.shape,test3.shape,test4.shape,custTotal_store4.shape

((400000, 2), (400000, 46), (400000, 321), (540651, 46))

In [381]:
custTotal_store4.columns,test3.columns

(Index([        u'Territory',          u'Business',        u'Store_Code',
              u'Customer_ID',       u'Nationality',      u'Income_Range',
                 u'Job_Type',    u'Marital_Status',            u'Gender',
                    u'State',          u'Language',    u'Loyalty_Status',
                      u'Age',            u'Points',     u'First_txn_day',
        u'First_txn_weekday',            u'Region',       u'Region_Code',
               u'Store_Name',         u'Mall_Name',      u'Store_Format',
            u'Sales_Per_Day',  u'Train_Test_Store',  u'Store_Size_Sq_Ft',
            u'Total_Revenue',      u'purchaseFlag',       u'revenuesize',
               u'revenuesum',        u'revenuemax',        u'revenuemin',
                 u'itemsize',           u'itemsum',           u'itemmax',
                  u'itemmin',           u'dissize',            u'dissum',
                   u'dismax',            u'dismin',         u'unitssize',
                 u'unitssum',         

In [379]:
neededColumns2=list(custTotal_store4.columns)

In [380]:
test3=test3[neededColumns2]

In [382]:
totaldata=custTotal_store4.append(test3)

In [358]:
test3.isnull().sum()

Territory            0
Business             0
Store_Code           0
Customer_ID          0
Nationality          0
Income_Range         0
Job_Type             0
Marital_Status       0
Gender               0
State                0
Language             0
Loyalty_Status       0
Age                  0
Points               0
First_txn_day        0
First_txn_weekday    0
Region               0
Region_Code          0
Store_Name           0
Mall_Name            0
Store_Format         0
Sales_Per_Day        0
Train_Test_Store     0
Store_Size_Sq_Ft     0
Total_Revenue        0
revenuesize          0
revenuesum           0
revenuemax           0
revenuemin           0
itemsize             0
itemsum              0
itemmax              0
itemmin              0
dissize              0
dissum               0
dismax               0
dismin               0
unitssize            0
unitssum             0
unitsmax             0
unitsmin             0
invoicesize          0
invoiceesum          0
invoicemax 

In [357]:
test3.dtypes

Territory             object
Business              object
Store_Code             int64
Customer_ID            int64
Nationality           object
Income_Range          object
Job_Type              object
Marital_Status        object
Gender                object
State                 object
Language              object
Loyalty_Status        object
Age                  float64
Points               float64
First_txn_day        float64
First_txn_weekday    float64
Region                object
Region_Code            int64
Store_Name            object
Mall_Name             object
Store_Format          object
Sales_Per_Day        float64
Train_Test_Store      object
Store_Size_Sq_Ft     float64
Total_Revenue        float64
revenuesize            int64
revenuesum           float64
revenuemax           float64
revenuemin           float64
itemsize               int64
itemsum                int64
itemmax                int64
itemmin                int64
dissize                int64
dissum        

In [359]:
char_cols2 = test3.dtypes.pipe(lambda x: x[x == 'object']).index

test4 = pd.get_dummies(test3,prefix=char_cols2)

((400000, 2), (400000, 45))

In [408]:
predValue=clf.predict(testNw.drop(['Store_Code','Customer_ID'],axis=1))

In [409]:
set(predValue)

{1}

In [178]:
custTrans.head()

,Territory,Business,Year,Week,Store_Code,City_Name,Store_Type,Transaction_Type,Return_Reason,Customer_ID,Invoices,Item_Count,Revenue,Discount,Units_Sold
0,United Arab Emirates,Max,2010,0,60065,Dubai,Stand alone,Return,Size Problem,1800000058056860,1,3,-169.0,0.0,-3
1,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000020917140,1,1,0.0,0.0,0
2,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000021265010,1,1,0.0,0.0,0
3,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000025693950,1,4,0.0,0.0,0
4,United Arab Emirates,Max,2016,0,60053,NaN,NaN,Purchase,NaN,1800000044104620,1,1,0.0,0.0,0


In [169]:
custTrans.columns

Index([u'Territory', u'Business', u'Year', u'Week', u'Store_Code',
       u'City_Name', u'Store_Type', u'Transaction_Type', u'Return_Reason',
       u'Customer_ID', u'Invoices', u'Item_Count', u'Revenue', u'Discount',
       u'Units_Sold'],
      dtype='object')

In [170]:
storeMas.columns

Index([u'Territory', u'Business', u'Region', u'Region_Code', u'Store_Code',
       u'Store_Name', u'Mall_Name', u'Store_Format', u'Sales_Per_Day',
       u'Store_Launch_Date', u'Train_Test_Store', u'Geo_Field',
       u'Store_Size_Sq_Ft', u'Customer_Count', u'Total_Revenue'],
      dtype='object')

In [157]:
transacCusts2=custTotal_store2['Customer_ID'].unique()
leftout=test[~test['Customer_ID'].isin(transacCusts2)]

In [158]:
leftout.shape

(0, 2)

In [152]:
test.shape

(400000, 2)